In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

2025-05-24 14:18:57.957395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748096337.990581    6828 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748096338.000667    6828 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-24 14:18:58.034473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout, BatchNormalization

def build_cnn_lstm_model():
    inputs = tf.keras.Input(shape=(24, 1))  # Shape: (timesteps, features)

    # CNN part
    x = layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # LSTM part
    x = layers.LSTM(64, return_sequences=True, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.LSTM(128, return_sequences=False, activation="relu")(x)  # Final LSTM layer
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Dense layers
    x = layers.Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(1)(x)  # Regression output (e.g., BPM)

    model = tf.keras.Model(inputs, outputs)
    return model


In [3]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [4]:
# Check TensorFlow version and GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
model=build_cnn_lstm_model()
# model1=build_cnn_lstm_model()

I0000 00:00:1748096343.137818    6828 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [6]:
import numpy as np
import pandas as pd


In [7]:
X=np.load("input.npy")
Y=np.load("out.npy")

In [8]:
X

array([[[117.54315336],
        [117.54315336],
        [117.2005136 ],
        ...,
        [117.29173759],
        [117.62220746],
        [117.62220746]],

       [[117.540685  ],
        [117.540685  ],
        [117.61534145],
        ...,
        [119.32458413],
        [119.57043436],
        [119.57043436]],

       [[119.91978626],
        [119.91978626],
        [119.79420653],
        ...,
        [121.21410458],
        [121.21410458],
        [121.21410458]],

       ...,

       [[102.29422963],
        [101.78716716],
        [101.78716716],
        ...,
        [100.57082101],
        [100.57082101],
        [ 99.99666251]],

       [[ 99.99666251],
        [ 99.24707409],
        [ 99.24707409],
        ...,
        [ 97.53497031],
        [ 98.18329085],
        [ 98.18329085]],

       [[ 98.18329085],
        [ 97.99323596],
        [ 97.99323596],
        ...,
        [ 99.94760233],
        [ 99.84978095],
        [ 99.84978095]]])

In [9]:
Y

array([[ 75],
       [ 75],
       [ 75],
       ...,
       [100],
       [ 99],
       [ 99]])

In [10]:
model.compile(optimizer="adam",metrics=["mae"],loss="mae")
# model1.compile(optimizer="adam",metrics=["mae"],loss="mae")

In [11]:
model.fit(X,Y,epochs=200,callbacks=callback,validation_split=0.3)
# model1.fit(X,Y,epochs=200,callbacks=callback,validation_split=0.2)

Epoch 1/200


I0000 00:00:1748096349.990816    6901 service.cc:148] XLA service 0x7ff88c014620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748096349.990879    6901 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-05-24 14:19:10.186899: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748096351.211809    6901 cuda_dnn.cc:529] Loaded cuDNN version 90300


 3/35 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 85.3319 - mae: 85.3319 

I0000 00:00:1748096358.024095    6901 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 ━━━━━━━━━━━━━━━━━━━━ 24s 330ms/step - loss: 82.1833 - mae: 82.1833 - val_loss: 88.4598 - val_mae: 88.4598
Epoch 2/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 32.8650 - mae: 32.8650 - val_loss: 63.2908 - val_mae: 63.2908
Epoch 3/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 12.5121 - mae: 12.5121 - val_loss: 47.6820 - val_mae: 47.6820
Epoch 4/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 12.8333 - mae: 12.8333 - val_loss: 45.6010 - val_mae: 45.6010
Epoch 5/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 15.9943 - mae: 15.9943 - val_loss: 26.8553 - val_mae: 26.8553
Epoch 6/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 11.2175 - mae: 11.2175 - val_loss: 29.3290 - val_mae: 29.3290


In [12]:
X.shape

(1560, 24, 1)

In [13]:
Y

array([[ 75],
       [ 75],
       [ 75],
       ...,
       [100],
       [ 99],
       [ 99]])

In [14]:
# model.save("bpm_model.keras")